In [1]:
import torch

import numpy as np
from main.models import ConvNet
from main.utils import save_experiment
from dataclasses import dataclass
from main.active_learning import run_active_learning
from main.prepare_data import create_repeated_MNIST_dataloaders

%reload_ext autoreload
%autoreload 2

c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set configurations
@dataclass
class ActiveLearningConfig:
    subset_of_weights: str = 'last_layer'
    hessian_structure: str = 'kron'
    backend: str = 'AsdlGGN'
    temperature: float = 1.0
    max_training_samples: int = 100
    acquisition_batch_size: int = 5
    al_method: str = 'max_logdet_S'
    test_batch_size: int = 512
    num_classes: int = 10
    num_initial_samples: int = 40
    training_iterations: int = 4096 * 6
    scoring_batch_size: int = 64
    train_batch_size: int = 64
    extract_pool: int = 0
    num_repeats: int = 10
    samples_per_digit: int = 50

experiment_name = '10x_redundant_logdet'  # provide descriptive name for the experiment

config = ActiveLearningConfig()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

save_results = True
num_runs = 5

Using device: cpu


In [3]:
for i in range(num_runs):
    # load data
    train_loader, test_loader, pool_loader, active_learning_data = create_repeated_MNIST_dataloaders(config)
    
    # get results
    results = run_active_learning(
        train_loader=train_loader,
        test_loader=test_loader, 
        pool_loader=pool_loader,
        active_learning_data=active_learning_data,
        model_constructor=ConvNet, 
        config=config, 
        device=device
        )

    # save results and configuration
    if save_results:
        experiment_id = experiment_name + '_' + str(i + 1)
        save_experiment(config, results, experiment_id)

c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\torch\utils\data\sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."
Training Set Size:  45%|████▌     | 45/100 [00:12<02:16,  2.47s/it]

Dataset indices:  [4415 2793 1076 4021  876]
Scores:  [0, 0, 0, 0, 0]
Labels:  tensor([9, 3, 6, 1, 1])


2024-07-29 12:10:19,831 - INFO - Training set size: 45, Test set accuracy: 71.49, Test set loss: -0.0177
Training Set Size:  50%|█████     | 50/100 [00:22<01:51,  2.23s/it]

Dataset indices:  [3206  263 4974   88 4617]
Scores:  [0, 0, 0, 0, 0]
Labels:  tensor([0, 4, 4, 3, 0])


2024-07-29 12:10:30,960 - INFO - Training set size: 50, Test set accuracy: 72.49, Test set loss: -0.0162
Training Set Size:  55%|█████▌    | 55/100 [00:33<01:40,  2.23s/it]

...

Dataset indices:  [1484  992 1906 3765 1775]
Scores:  [0, 0, 0, 0, 0]
Labels:  tensor([2, 6, 0, 1, 1])


2024-07-29 12:21:53,136 - INFO - Training set size: 95, Test set accuracy: 80.34, Test set loss: -0.0250
Training Set Size: 100%|██████████| 100/100 [02:13<00:00,  2.25s/it]

Dataset indices:  [ 171 2614 4257 2134  281]
Scores:  [0, 0, 0, 0, 0]
Labels:  tensor([8, 8, 5, 1, 9])


2024-07-29 12:22:03,958 - INFO - Training set size: 100, Test set accuracy: 81.57, Test set loss: -0.0244
Training Set Size: 100%|██████████| 100/100 [02:24<00:00,  2.41s/it]


Experiment saved in experiments\10x_redundant_random_5
